### This notebook demonstrates the capabilities of the DeterministicReranking algorithm.
The algorithm provides a way to construct balanced rankings of candidates based on precalculated scores.

*Based on: [Sahin Cem Geyik, Stuart Ambler, & Krishnaram Kenthapadi (2019). Fairness-Aware Ranking in Search & Recommendation Systems with Application to LinkedIn Talent Search. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining.](https://doi.org/10.48550/arXiv.1905.01989)*

#### 1. Introduction

Ranking algorithms are at the core of search and recommendation systems used in, among others, hiring, college admissions, and web-searches. It is clear that algorithmic bias in such cases can amplify unacceptable discrimination based on race, gender, or other so-called **protected attributes**. Thus, a way of constructing "fair" rankings is needed.

Algorithms presented here take as input a **dataset that has already been ranked** (scored), possibly by some other machine learning model, and order them in a way that satisfies specified **fairness requirements**, expressed in a form of a **distribution over the protected attributes**.

#### 2. A toy example

Let's say we have a collection of 10 balls: 5 red and 5 blue. We assign each of them a **score from 0 to 100**; however, for some reason the red balls have a **higher average score** than the blue ones.

In [20]:
import numpy as np
import pandas as pd

balls = pd.DataFrame([['r', 100],['r', 90],['r', 85],['r', 70],['b', 70],['b', 60],['b', 50],['b', 40],['b', 30],['r', 20]],
                     columns=['color', 'score'])

print(f"Red mean score: {np.mean(balls[balls['color'] == 'r']['score'])}")
print(f"Blue mean score: {np.mean(balls[balls['color'] == 'b']['score'])}")

Red mean score: 73.0
Blue mean score: 50.0


Now, say we want to take the **5 best balls** based on their score. In real life, the need for this limited "sub-ranking" may arise for many reasons. For example, the landing page of our ball-selling website may only have space for 5 items. 

This is similar to the case presented in the original paper, where the algorithm is used to rank LinkedIn users for recruiters.

In [21]:
balls.sort_values(by='score', ascending=False)[:5]

,color,score
0,r,100
1,r,90
2,r,85
3,r,70
4,b,70


Of course, we notice that we have only 1 blue ball in this ranking, and it is in the last position! On one hand, it seems fair in terms of scores. However, we may want a more **equal representation** of different colors in the ranking. The reasons for that in real life (e.g. representation in government) are clear.

In our case, the color is the **protected attribute**, and the red balls are a **privileged class**.

We may get a fairer ranking using the `DeterministicReranking` class:

In [22]:
from aif360.datasets import RegressionDataset
import sys
sys.path.append("../aif360/algorithms/postprocessing/")
from deterministic_reranking import DeterministicReranking

# Initialize a RegressionDataset with color as the protected attribute and red as the privileged class.
balls_ds = RegressionDataset(df=balls, dep_var_name='score', protected_attribute_names=['color'], privileged_classes=[['r']])
# keep the un-normalized scores for clarity; RegressionDataset normalizes them to be from 0 to 1.
balls_ds.labels = np.transpose([balls['score']])

To initialize the DeterministicReranking class, we need to pass the **protected attribute values** of the priviliged and unprivileged groups.
We do that using a list of dictionaries for each group, with the **name of the attribute as the key**. 

In [23]:
# The RegressionDataset class automatically maps the priviliged attribute value (color='red') to 1 and the other to 0.
dr = DeterministicReranking(unprivileged_groups=[{'color': 0}], privileged_groups=[{'color': 1}])

Use the `fit_predict` method to get the ranking. The arguments are:
- `dataset` is the dataset to construct a ranking from;
- `rec_size` is the **size** of the ranking we need - in our case 5;
- `target_prop` is the **desired proportion** of items of each group in the ranking in the form of dictionary; the keys are the corresponding protected attribute values. We need equal representation, so we pass `{0: 0.5, 1: 0.5}`;
- `rerank_type` is the algorithm to use. For now, we stick to the default `Constrained`. For further details, skip to HERE;
- `renormalize_scores` will normalize the scores in the result so that the lowest is 0 and the highest is 1. Default is `False`.

In [24]:
fair_rank = dr.fit_predict(dataset=balls_ds, rec_size=5, target_prop={0: 0.5, 1: 0.5})
fair_rank.convert_to_dataframe()[0]

,color,score
0,1.0,100.0
1,1.0,90.0
4,0.0,70.0
2,1.0,85.0
5,0.0,60.0
6,0.0,50.0


In this result, the proportions are equal. Additionally, as the algorithms goes through positions in the ranking one-by-one, checking each time for violations of fairness, the items belonging to the unprivileged group (blue balls) **aren't all at the "bottom"** of the ranking.

#### Application to the Law School GPA Dataset

We now apply the algorithm to a more serious example - the Law School GPA dataset.

The aim is to obtain a ranking of size 20 according to the value in `zfygpa` with fairness constraints on `race`.

In [25]:
import sys
from aif360.datasets import LawSchoolGPADataset

dataset = LawSchoolGPADataset()

First, we take a ranking strictly according to the score.

In [33]:
df = dataset.convert_to_dataframe()[0].sort_values(by=['zfygpa'], ascending=False)
ranked_dataset = RegressionDataset(df=df, dep_var_name='zfygpa', protected_attribute_names=['race'], privileged_classes=[[1]])
# Again, for clarity we leave the labels (scores) unnormalized.
ranked_dataset.labels = np.transpose([df['zfygpa']])

Problem: **all the items come from the priviliged group!**

In [27]:
rank = ranked_dataset.convert_to_dataframe()[0][:20]
rank

,lsat,ugpa,race,zfygpa
5030,0.716216,1.000,1.0,1.000000
5924,0.675676,0.750,1.0,0.970960
14599,0.891892,1.000,1.0,0.960859
10311,0.783784,0.700,1.0,0.953283
9635,0.783784,0.750,1.0,0.953283
3924,0.837838,0.875,1.0,0.952020
6723,0.891892,0.950,1.0,0.948232
6399,0.729730,0.525,1.0,0.944444
3070,1.000000,0.975,1.0,0.944444
9533,0.675676,1.000,1.0,0.930556


Try to get a fair ranking using `DeterministicReranking`:

In [28]:
dr = DeterministicReranking(unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
fair_rank = dr.fit_predict(dataset, rec_size=20, target_prop={0: 0.5, 1: 0.5})
fair_rank.convert_to_dataframe()[0]

,lsat,ugpa,race,zfygpa
5030,0.637931,1.000000,1.0,1.000000
5924,0.586207,0.473684,1.0,0.970960
2403,0.000000,0.842105,0.0,0.900253
14599,0.862069,1.000000,1.0,0.960859
3492,0.689655,0.421053,0.0,0.888889
10311,0.724138,0.368421,1.0,0.953283
4925,0.620690,0.105263,0.0,0.888889
9635,0.724138,0.473684,1.0,0.953283
8769,0.310345,0.526316,0.0,0.842172
3924,0.793103,0.736842,1.0,0.952020


This seems much better.

To get a better idea of the level of fairness, we can quantify it using the **Infeasible Index** metric, which measures the number of indices at which the fairness requirements are violated. For a ranking of size $r$ and groups of items $g_i \in G$ with desired proportions $p_i$ it is defined as follows:

$$InfeasibleIndex_r = \sum_{k=0}^{r} 1(\exists\ group\ g_i \in G: count_k(g_i) < \lfloor(p_i*k)\rfloor)$$

Calculate the InfeasibleIndex using the `infeasible_index` method of the `RegressionDatasetMetric` class, first of the strictly score-based ranking, and then of the fair ranking.

It returns the value of the metric, as well as positions in the ranking at which the fairness constraints are violated.

In [29]:
sys.path.append("../aif360/metrics/")
from regression_metric import RegressionDatasetMetric

In [30]:
m = RegressionDatasetMetric(dataset=ranked_dataset, unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
m.infeasible_index(target_prop={0: 0.5, 1: 0.5}, r=20)

(18, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

In [31]:
# The II for this ranking is much better!
m_fair = RegressionDatasetMetric(dataset=fair_rank, unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
m_fair.infeasible_index(target_prop={0: 0.5, 1: 0.5}, r=20)

(9, [1, 3, 5, 7, 9, 11, 13, 15, 17])

We can now quantify the loss in the quality of the ranking (how "out of order" the items are, based on their scores) induced by the reranking algorithm. For this we can use the **Discounted Cumulative Gain**, defined as follows:

$$DCG@k = \sum_{j=0}^{k} \frac{score(j)}{log_2(j+1)}$$

where $score(j)$ denotes the score of the item at position $j$.

This metric can also be normalized against the value of the "perfect" strictly score-based ordering, giving it a range from 0 to 1.

In [32]:
print(f'Normalized DCG of strictly score-based ranking: {m.discounted_cum_gain(normalized=True)}')
print(f'Normalized DCG of fair ranking: {m_fair.discounted_cum_gain(normalized=True)}')

Normalized DCG of strictly score-based ranking: 1.0
Normalized DCG of fair ranking: 0.9631630347859398
